<h4>连接类型</h4>

- inner join, 内部连接(保留左,右数据集内某个键都存在的行)
- outer join, 外部连接(保留左侧或右侧数据集中具有某个键的行)
- left outer join, 左外部连接(保留左侧数据集中具有某个键的行)
- right outer join, 右外部连接
- left semi join, 左半连接(如果某键在右侧数据行中出现, 则保留且仅保留左侧数据行)
- left anti join, 左反连接(如果某键在右侧数据行中没出现, 则保留且仅保留左侧数据行)
- natural join, 自然连接(通过隐式匹配两个数据集之间相同名称的列来执行连接)
- cross join, 笛卡尔连接

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python").getOrCreate()

<h4>首先创建一些简单数据集</h4>

In [7]:
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Mateei", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])
]).toDF("id", "name", "graduate_program", "spark_status")

In [19]:
%%html
<img src="img/createDataFrame.png", width=400>

In [6]:
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkley"),
    (2, "Masters", "EECS", "UC Berkley"),
    (1, "Ph.D.", "EECS", "UC Berkley")
]).toDF("id", "degree", "department", "school")

In [8]:
sparkStatus = spark.createDataFrame([
    (500, "Vice PPresident"),
    (250, "PMC Member"),
    (100, "Contributor")
]).toDF("id", "status")

In [10]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

<h4>内连接</h4>

- 内连接是默认的连接操作, 只有指定键相等,才会将两行拼接起来返回
- 对左表使用JOIN方法, 方法内三个参数: 右表, 连接键, 连接方式
- A.join(B, joinExpression, joinType)

In [25]:
%%html
<img src="img/innerjoin2.png", width=600>

-- in SQL<br>
SELECT * FROM person JOIN graduateProgram<br>
ON person.graduate_program = graduateProgram.id

<h4>外连接</h4>

- joinType: outer
- 返回所有表的所有行
- 如果两行的指定键不相等, 则会将左侧或右侧DataFrame中没有匹配的行的各列用null替换

<h4>左外连接</h4>

- joinType: left_outer
- 返回左表所有行, 并连接右边有匹配项的行
- 对于右表无法匹配的行, Spark将插入null

<h4>右外连接</h4>

- joinType: right_outer

<h4>左半连接</h4>

- joinType: left_semi
- 不包含右侧DataFrame的任何值, 只是查看左侧DataFrame是否存在于右侧DataFrame中
- 左半连接可以看作是DataFrame的过滤器

In [26]:
%%html
<img src="img/leftsemi.png", width=600>

<h4>左反连接</h4>

- joinType: left_anti
- 左反连接(left anti join)与左半连接(left semi join)相反
- 左表会只保留右表没有的键

-- in SQL<br>
SELECT * FROM graduateProgram LEFT ANTI JOIN person<br>
ON graduateProgram.id = person.graduate_program

<h4>交叉连接(笛卡尔积连接)</h4>

- 交叉连接: cross join, 笛卡尔积连接: cartesian product
- 交叉连接会将左右两表每一行进行连接, 最产生大量行数
- person.crossJoin(graduateProgram).show()
- SELECT * FROM graduateProgram CROSS JOIN 

<h4>对复杂类型的连接操作</h4>

- 任何返回boolean值得表达式都是有效的连接表达式
- 将person的id命名为personId后, 判断sparkStatus的id是否在spark_status中

In [29]:
%%html
<img src="img/complexjoin.png", width=600>

<h4>处理重复列名</h4>

- 在连接两表后, 可能会出现两个拥有相同列名的列
- 比如有两个id列, 这时候选择id就会报错

In [30]:
%%html
<img src="img/dup.png", width=600>

- 方法一
    - 采用不同的连接表达式, 也就是从布尔表达式更改为字符串
- 方法二
    - 连接后删除列
- 方法三
    - 重命名列

In [35]:
%%html
<img src="img/stringjoin.png", width=300>

In [36]:
%%html
<img src="img/dropjoin.png", width=800>

<h4>Spark如何执行连接</h4>

- 首先要了解两个起作用的核心模块
- 点对点通信模式 (node-to-node communication strategy)
- 逐点计算模式 (per node computation strategy)

<h4>通信策略</h4>

- Spark使用两种不同的方式处理集群通信问题
- all-to-all通信的shuffle join
- broadcast

<h4>大表与大表连接</h4>

- 当用一个大表连接一个大表时,最终就是一个shuffle join
- 每个节点会与所有其他节点通信, 并根据哪个节点具有某个键来共享数据,
- 在数据没有合理分区的情况下, 网络会因通信量会阻塞,连接会很耗时

<h4>大表与小表连接</h4>

- 当表的大小足够小以便能够放入单个节点内存中且还有空闲空间时, 可以优化连接
- 这时候我们使用broadcast join, 把数据较小的表复制到集群所有节点
- 只需要在开始时执行一次,能够避免all-to-all通信
- 每个工作节点独立执行作业, 无需等待其他工作节点
- 可以指定对小表使用broadcast

In [38]:
%%html
<img src="img/broadcast.png", width=600>